In [1]:
import sqlite3
import pandas as pd
import sys
if (sys.version_info[0] == 3):
    from urllib.request import urlopen
else:
    from urllib2 import urlopen
from lxml import etree
import numpy as np

In [2]:
conn = sqlite3.connect('caves.db')
query = "SELECT * FROM caves ORDER BY date_creation"
#pd.read_sql_query(query,conn, parse_dates=['date_creation', 'date_suppression'])

In [3]:
def infos_annonce2(url_request):
    htmlparser = etree.HTMLParser()
    page_ads = urlopen(url_request)
    tree_ads = etree.parse(page_ads, htmlparser)
    ad_title = tree_ads.xpath("/html/body/section[1]/main/section/section/section/header/h1")[0].text

    results = []
    for i_section in range(1, 3):
        for i_div in range(1, 15):
            xpath_category_i = "/html/body/section[1]/main/section/section/section/section/section[" + str(i_section) + "]/div[" + str(i_div) + "]/h2/span[" + str(1) + "]"
            xpath_value_i = "/html/body/section[1]/main/section/section/section/section/section[" + str(i_section) + "]/div[" + str(i_div) + "]/h2/span[" + str(2) + "]"
            try:
                category_i = tree_ads.xpath(xpath_category_i)[0].text.replace("\\n", "")
                value_i = tree_ads.xpath(xpath_value_i)[0].text
            except:
                category_i = ''
                value_i = ''
            if(len(category_i) != 0):
                results.append([i_section, i_div, category_i, value_i])
    
    # on cherche les informations qui nous intéressent
    # la surface
    category = "Surface"
    result = ""
    for line in results:
        if(line[2].find(category) > -1):
            result = line[3]
    if(len(result) > 0):
        result = result.split(" ")[0]
    if(len(result)>0):
        ad_surface = int(result)
    else:
        ad_surface = 0

    # le nombre de pièces
    category = "Pièces".decode('utf8')
    result = ""
    for line in results:
        text = line[2]#.decode('latin1')
        if(text.find(category) > -1):
            result = line[3]
    if(len(result) > 0):
        result = result.split(" ")[0]
    if(len(result)>0):
        pieces = int(result)
    else:
        pieces = 0

    # pour ne pas garder les appartements
    if(pieces > 1):
        ad_surface = np.nan

    # on récupère la description
    ad_description = ""
    for section_i in range(1, 3):
        for div_i in range(10, 15):
            #xpath1 = "//*[@id=\"adview\"]/section/section/section/div[12]/p[2]/text()[" + str(i) + "]"
            xpath = "/html/body/section[1]/main/section/section/section/section/section[" + str(section_i) + "]/div[" + str(div_i) + "]/p[2]/text()"
            #tmp = tree_ads.xpath(xpath1)[0].replace("\\n", "")
            #ad_description = ad_description + "<br>" + tmp
            try:
                obj = tree_ads.xpath(xpath)
                for i in range(len(obj)):
                    ad_description = ad_description + "<br>" + obj[i].replace("\\n", "")
            except:
                print("description not found")
    ad_description = ad_description[4:]

    # on récupère la date de création de l'annonce
    xpath = "//*[@id=\"adview\"]/section/section/section/p"
    ad_creation = tree_ads.xpath(xpath)[0].values()[2]

    # on cherche des mots clés pour voir s'il s'agit d'un appartement
    # si c'est le cas, on fixe la surface à 0 pour ne pas le prendre en compte
    keywords = ['Appart', 'appart', 'studio', 'Studio', 'meublé'.decode('utf8')]
    for keyword in keywords:
        if ad_title.find(keyword) != -1:
            ad_surface = np.nan


    print("surface : " + str(ad_surface))
    return ad_surface, ad_description, ad_creation

In [4]:
url = "https://www.leboncoin.fr/locations/1328164479.htm?ca=12_s"
ad_surface, ad_description, ad_creation = infos_annonce2(url)
df_ajouts = pd.DataFrame()
df_ajouts = df_ajouts.append({'id':123456, 'title':'tiiitre', 'price':127, 'url':url, 'type':'LBC', 'city':'Paris', 'surface':ad_surface, 'description':ad_description, 'date_creation':ad_creation, 'date_suppression':np.nan}, ignore_index=True)
df_ajouts = df_ajouts.append({'id':123456, 'title':'tiiitre', 'price':127, 'url':url, 'type':'LBC', 'city':'Paris', 'surface':ad_surface, 'description':ad_description, 'date_creation':ad_creation, 'date_suppression':np.nan}, ignore_index=True)
#conn.commit()
df_ajouts.to_sql("caves", conn, if_exists='append', index=False)


surface : 3


In [5]:
#cur.close()
conn.close()

In [6]:
df_ajouts

,city,date_creation,date_suppression,description,id,price,surface,title,type,url
0,Paris,2017-10-30,NaN,"Bonjour,<br>Louez une cave de 2,60m² (ou 6,20m...",123456.0,127.0,3.0,tiiitre,LBC,https://www.leboncoin.fr/locations/1328164479....
1,Paris,2017-10-30,NaN,"Bonjour,<br>Louez une cave de 2,60m² (ou 6,20m...",123456.0,127.0,3.0,tiiitre,LBC,https://www.leboncoin.fr/locations/1328164479....


In [8]:
df = df_ajouts[['url', 'id']]

In [16]:
import datetime
i = datetime.datetime.now()
t = str(i.year) + "-" + str(i.month) + "-" + str(i.day)
t

'2017-10-30'

In [21]:
tab = [t for i in range(df.shape[0])]
df['date_mesure'] = tab

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
df

,url,id,date_mesure
0,https://www.leboncoin.fr/locations/1328164479....,123456.0,2017-10-30
1,https://www.leboncoin.fr/locations/1328164479....,123456.0,2017-10-30


In [23]:
str(datetime.datetime.now())

'2017-10-30 13:32:09.951616'